In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/open_data')

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
torch.manual_seed(1015)
# define 'device' to upload tensor in gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_csv("train.csv", encoding = 'euc-kr')

In [5]:
train = pd.read_csv("train.csv", encoding = 'euc-kr')
train['DateTime'] = pd.to_datetime(train.DateTime)
train['date'] = train.DateTime.dt.date
train  = train.groupby('date').sum().reset_index()

In [6]:
# scaling
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 4))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 4))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values

In [7]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            batch_first=True)
        self.hidden_lstm = nn.LSTM(input_size = hidden_size,
                                   hidden_size = hidden_size,
                                   batch_first=True)
        
        self.time_fc = nn.Linear(hidden_size, 4)
    
    def forward(self, x_time):
    
        out_time, _ = self.lstm(x_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        out_time, _ = self.hidden_lstm(out_time)
        
        
        out_time = self.time_fc(out_time[:,-7:, :])
        
        return out_time.view(-1,7,4)
    
model = LSTM(input_size = 4, hidden_size = 30).to(device)

In [8]:
window_x = torch.tensor(window_x).float().to(device)
window_y = torch.tensor(window_y).float().to(device)

# Train model
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
criterion = nn.MSELoss(size_average = True)
num_epochs  = 700
train_error = []
for t in range(num_epochs):
    train_pred = model(window_x)
    loss = criterion(train_pred, window_y) ### trend
    train_error.append(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if t % 10 == 0 and t !=0:
        print(f"{t} Epochs train MSE: {loss.item():1.5f}")

10 Epochs train MSE: 0.02835
20 Epochs train MSE: 0.01247
30 Epochs train MSE: 0.01229
40 Epochs train MSE: 0.01079
50 Epochs train MSE: 0.00841
60 Epochs train MSE: 0.00808
70 Epochs train MSE: 0.00795
80 Epochs train MSE: 0.00784
90 Epochs train MSE: 0.00775
100 Epochs train MSE: 0.00765
110 Epochs train MSE: 0.00749
120 Epochs train MSE: 0.00721
130 Epochs train MSE: 0.00660
140 Epochs train MSE: 0.00675
150 Epochs train MSE: 0.00647
160 Epochs train MSE: 0.00624
170 Epochs train MSE: 0.00608
180 Epochs train MSE: 0.00599
190 Epochs train MSE: 0.00591
200 Epochs train MSE: 0.00580
210 Epochs train MSE: 0.00564
220 Epochs train MSE: 0.00563
230 Epochs train MSE: 0.00667
240 Epochs train MSE: 0.00632
250 Epochs train MSE: 0.00585
260 Epochs train MSE: 0.00560
270 Epochs train MSE: 0.00547
280 Epochs train MSE: 0.00536
290 Epochs train MSE: 0.00544
300 Epochs train MSE: 0.00531
310 Epochs train MSE: 0.00515
320 Epochs train MSE: 0.00500
330 Epochs train MSE: 0.00472
340 Epochs train MS

In [9]:
submission = pd.read_csv("submission.csv", encoding = 'euc-kr')
last_month = torch.tensor(window_x[-1,:,:][np.newaxis,...]).float().to(device)
for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model(last_month)
    last_month = torch.cat([last_month[-7:], next_week], axis = 1)
    
    pred_week = next_week.cpu().detach().numpy().reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission

,DateTime,사용자,세션,신규방문자,페이지뷰
0,2020-11-09,1931,1817,389,47312
1,2020-11-10,4689,4708,1269,126655
2,2020-11-11,5305,5285,1478,141225
3,2020-11-12,4560,4553,1163,113556
4,2020-11-13,4184,4151,1079,107849
...,...,...,...,...,...
56,2021-01-04,2316,2217,518,58224
57,2021-01-05,2841,2742,682,65283
58,2021-01-06,2715,2675,682,65645
59,2021-01-07,2138,2120,526,51526


In [10]:
submission.to_csv('submission.csv', index = False, encoding = 'euc-kr')